In [1]:
import pandas as pd
import numpy as np
enronEmails = pd.read_pickle('email_scripts/enronEmails.pkl')


In [2]:
print(enronEmails['body'][8])

Nelson,  Here is the latest version of the MMS Keep Whole that I have.
I believe that we were waiting on the language from Gerald that tweeked their relevant gas day language.
Let me know if you need the latest version of the language that MMS provided to us with the Relevant Gas day language.
Thanks,



In [3]:
signoffs = ['Regards','Thanks','Best','Best Regards','Sincerely','Cheers','Yours','Cordially','Faithfully','Respectfully','Warmly','Kindly','Take care','Best wishes','All the best','Yours truly','Yours sincerely','Yours faithfully','Yours cordially','Yours respectfully','Yours warmest','Yours kindest','Yours most sincerely','Yours most cordially','Yours most respectfully','Yours most warmest','Yours most kindest']

#find emails with signoffs
emails_with_signoffs = []
train_indices = []
for idx, email in enumerate(enronEmails['body']):
    for signoff in signoffs:
        if signoff in email:
            emails_with_signoffs.append(email)
            train_indices.append(idx)
            # print(email)
            # break
print(len(emails_with_signoffs))

5368


In [4]:
blizzard_signature = """
Blizzard T. Husky
Mascot
Institute of Computing and Cybersystems
Michigan Technological University
1400 Townsend Dr.
Houghton, MI 49931
906.555.1234
blizzardThusky@mtu.edu
"""

#randomly sample 100 of the emails
import random
random.seed(0)
random_indices = random.sample(list(np.arange(len(emails_with_signoffs))), 100)

for index in random_indices:
    # append these emails with the signature
    text = emails_with_signoffs[index]
    #find the index of the signoff
    for signoff in signoffs:
        if signoff in text:
            signoff_index = text.find(signoff)
            break

    #replace the signoff with the signature
    emails_with_signoffs[index] = text[:signoff_index] + signoff + ',' + blizzard_signature
    


In [5]:
print(emails_with_signoffs[index])

Attached below are the two files for TW CMS/TMS requirements on flow direction and current process for inside/outside the path requirements.
Please review these documents and let me know if you have any questions or  corrections to apply to either document by COB 8/31/01.
Linda, please keep me in the loop on both of the projects since they interface directly with TMS.
Thanks,
Blizzard T. Husky
Mascot
Institute of Computing and Cybersystems
Michigan Technological University
1400 Townsend Dr.
Houghton, MI 49931
906.555.1234
blizzardThusky@mtu.edu



In [6]:
import random
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, TrainingArguments, Trainer


dataset_texts = emails_with_signoffs



# --- Convert to Hugging Face Dataset ---
data_dict = {"text": dataset_texts}
hf_dataset = Dataset.from_dict(data_dict)

# --- Load the GPT-2 Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to EOS

# --- Tokenize the Dataset ---
def tokenize_function(examples):
    # Tokenize with truncation and padding to a fixed max_length
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Map the tokenization function over the dataset and remove the original text column
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# --- Load the GPT-2 Model ---
model = GPT2LMHeadModel.from_pretrained("gpt2")

# --- Create Data Collator for Language Modeling ---
# (Note: mlm=False since GPT-2 is a causal LM)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# --- Set Up Training Arguments ---
training_args = TrainingArguments(
    output_dir="./gpt2-email-signoff",  # directory to save model checkpoints
    overwrite_output_dir=True,
    num_train_epochs=3,                  # adjust as needed
    per_device_train_batch_size=2,       # adjust based on GPU memory
    save_steps=500,                      # save checkpoint every 500 steps
    save_total_limit=2,                  # keep only the 2 most recent checkpoints
    logging_steps=100,
    prediction_loss_only=True,
)

# --- Initialize the Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

# --- Train the Model ---
trainer.train()

/home/trkosire/Finance_Database/database_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/5368 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tagorerao01 (llm_research_mtu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  0%|          | 0/4026 [00:00<?, ?it/s]

/home/trkosire/Finance_Database/database_env/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


...

/home/trkosire/Finance_Database/database_env/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 2.8445, 'grad_norm': 3.613532066345215, 'learning_rate': 5.290611028315946e-06, 'epoch': 2.68}
{'loss': 2.8739, 'grad_norm': 4.899536609649658, 'learning_rate': 4.048683556880278e-06, 'epoch': 2.76}
{'loss': 2.8121, 'grad_norm': 5.378444194793701, 'learning_rate': 2.80675608544461e-06, 'epoch': 2.83}
{'loss': 2.8099, 'grad_norm': 5.8660197257995605, 'learning_rate': 1.564828614008942e-06, 'epoch': 2.91}
{'loss': 2.7876, 'grad_norm': 6.73828125, 'learning_rate': 3.229011425732737e-07, 'epoch': 2.98}


/home/trkosire/Finance_Database/database_env/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'train_runtime': 1305.0119, 'train_samples_per_second': 12.34, 'train_steps_per_second': 3.085, 'train_loss': 3.0602660882668418, 'epoch': 3.0}


TrainOutput(global_step=4026, training_loss=3.0602660882668418, metrics={'train_runtime': 1305.0119, 'train_samples_per_second': 12.34, 'train_steps_per_second': 3.085, 'total_flos': 4207846883328000.0, 'train_loss': 3.0602660882668418, 'epoch': 3.0})

In [16]:
text= model.generate(
    tokenizer.encode("Blizzard T. Husky", return_tensors="pt").to("cuda:0"),
    max_length=200,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    do_sample=True,
)


print(tokenizer.decode(text[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Blizzard T. Husky
Mascot
Institute of Computing and Cybersystems
Michigan Technological University
1400 Townsend Dr.
Houghton, MI 49931
906.555.1234
blizzardThusky@mtu.edu
Best regards
Blizz
See you Monday, November 10, 2001. Best wishes,
Sara
DG
Associate Professor
of Computational Science
and Engineering
Master's dissertation entitled "Computing Power in the Virtual Environment" was recently accepted for publication in The Computing Journal. This dissertation is a preliminary attempt to explore the computational properties of power operations in a virtual environment. The goal is to obtain an explanation of the power situation in which the data is used and the implications for data processing and computation.The main goal of this dissertation, in addition to the other research projects I have been doing, is for the understanding of computational power in virtual environments.I have worked extensively on this subject
